In [1]:
import time

import numpy as np

import serial
import serial.tools.list_ports

#import bokeh.plotting
#import bokeh.io
#bokeh.io.output_notebook()

In [2]:
HANDSHAKE = 0
TEMP_REQUEST = 1

In [3]:
def find_arduino(port=None):
    """Get the name of the port that is connected to Arduino."""
    if port is None:
        ports = serial.tools.list_ports.comports()
        for p in ports:
            if p.manufacturer is not None and "Arduino" in p.manufacturer:
                port = p.device
    return port


def handshake_arduino(
    arduino, sleep_time=1, print_handshake_message=False, handshake_code=0
):
    """Make sure connection is established by sending
    and receiving bytes."""
    # Close and reopen
    arduino.close()
    arduino.open()

    # Chill out while everything gets set
    time.sleep(sleep_time)

    # Set a long timeout to complete handshake
    timeout = arduino.timeout
    arduino.timeout = 2

    # Read and discard everything that may be in the input buffer
    _ = arduino.read_all()

    # Send request to Arduino
    arduino.write(bytes([handshake_code]))

    # Read in what Arduino sent
    handshake_message = arduino.read_until()

    # Send and receive request again
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()

    # Print the handshake message, if desired
    if print_handshake_message:
        print("Handshake message: " + handshake_message.decode())

    # Reset the timeout
    arduino.timeout = timeout

In [79]:
port = find_arduino()
arduino = serial.Serial(port, baudrate=115200)
handshake_arduino(arduino, handshake_code=HANDSHAKE, print_handshake_message=True)

Handshake message: Message received.



In [11]:
# Ask Arduino for data
arduino.write(bytes([TEMP_REQUEST]))

# Receive data
raw = arduino.read_until()

# Look at what we got
raw

raw_str = raw.decode()

# Take a look
raw_str

raw_str.rstrip()

'25.6699520000'

In [15]:
def getTEMP(arduino):
    arduino.write(bytes([TEMP_REQUEST]))
    raw = arduino.read_until()
    raw_str = raw.decode()
    return float(raw_str.rstrip())

In [16]:
getTEMP(arduino)

25.566681

In [18]:
getTEMP(arduino)

28.490265

In [22]:
def parse_raw(raw):
    """Parse bytes output from Arduino."""
    raw = raw.decode()
    if raw[-1] != "\n":
        raise ValueError(
            "Input must end with newline, otherwise message is incomplete."
        )

    t = raw.rstrip()
    return float(t)

In [23]:
def request_single_temp(arduino):
    """Ask Arduino for a single data point"""
    # Ask Arduino for data
    arduino.write(bytes([TEMP_REQUEST]))

    # Read in the data
    raw = arduino.read_until()

    # Parse and return
    return parse_raw(raw)

In [82]:
request_single_temp(arduino)

26.707062

In [77]:
arduino.close()

In [91]:
def get_timestamp():
    current_time = datetime.now()
    timestamp = current_time.timestamp()
    
    date_time = datetime.fromtimestamp(timestamp)
    str_date_time = date_time.strftime("%H:%M:%S")
    return str_date_time,current_time.time()

In [93]:
get_timestamp()[1]

datetime.time(16, 23, 8, 318087)